In [1]:
from equations import SEIR
from scipy.integrate import odeint
from scipy.optimize import curve_fit
import datetime as dt
import numpy as np
import pandas as pd
import altair as alt
import requests

In [2]:
germany_json = requests.get("https://ersatzworld.net/corona/data/Germany.json").json()
data = pd.DataFrame(np.stack([
        [dt.datetime.strptime(d, "%Y-%m-%d") for d in germany_json["dates"]], 
        germany_json["total"]
    ], axis=1), 
    columns=("Tag", "Fallzahl"))
data.tail(4)

,Tag,Fallzahl
74,2020-04-05,100123
75,2020-04-06,103374
76,2020-04-07,107663
77,2020-04-08,113296


In [3]:
data = data[data["Tag"] >= dt.datetime(year=2020, month=2, day=27)].reset_index(drop=True)

it1 = dt.datetime(year=2020, month=3, day=20)
it2 = dt.datetime(year=2020, month=3, day=28)
it3 = dt.datetime(year=2021, month=1, day=1)
it4 = dt.datetime(year=2021, month=4, day=1)
cases = np.array(data["Fallzahl"])
bev_de = germany_json["population"]
times = np.arange(0., len(cases), 1.0)
model = SEIR(p0=(bev_de-data["Fallzahl"][0], 0, data["Fallzahl"][0], 0),
             intervention_times=[(it1-data["Tag"][0]).days, 
                                 (it2-data["Tag"][0]).days, 
                                 (it3-data["Tag"][0]).days, 
                                 (it4-data["Tag"][0]).days],
             t_vals=times)

params, _ = curve_fit(
    model, 
    xdata=times, 
    ydata=cases,
    p0=[3., 2., 1., 0.],
    bounds=(
        [0., 0., 0., 0.],
        [10., 10., 10., bev_de]
    ))
params

array([ 3.4117278 ,  2.11355952,  1.03675675, 78.54508652])

In [4]:
r0, r1, r2, e0 = params
prediction_days = 5
times = np.arange(0., len(cases)+prediction_days, 1.0)
seir_predictions = model.getSEIR(times, [r0, r1, r2, r1, r0], e0)
sum_predictions = np.sum(seir_predictions[:, 1:], axis=1)

def clip_values(v):
    max_val = float(data.tail(1)["Fallzahl"]) * 1.1
    return np.where(np.less_equal(v, max_val), v, None)

dates = [data["Tag"][0] + dt.timedelta(days=i) for i in times]
seir_data = pd.DataFrame(np.concatenate([
    np.concatenate([np.stack([dates, clip_values(seir_predictions[:, i]), [t]*len(times)], axis=1)
                    for i, t in enumerate(["Susceptible", "Exposed", "Infectious", "Removed"])]),
    np.stack([dates, clip_values(sum_predictions), ["simulierte Summe"]*len(times)], axis=1)]),
    columns=("Tag", "Fallzahl", "Typ")
)
data["Typ"] = "gezählte Fälle"
seir_data = seir_data.append(data, ignore_index=True)
seir_data

,Tag,Fallzahl,Typ
0,2020-02-27,None,Susceptible
1,2020-02-28,None,Susceptible
2,2020-02-29,None,Susceptible
3,2020-03-01,None,Susceptible
4,2020-03-02,None,Susceptible
...,...,...,...
272,2020-04-04,96092,gezählte Fälle
273,2020-04-05,100123,gezählte Fälle
274,2020-04-06,103374,gezählte Fälle
275,2020-04-07,107663,gezählte Fälle


In [5]:
line = alt.Chart(seir_data).mark_line(point=False).encode(
    alt.X("Tag", title="Tag"),
    alt.Y("Fallzahl:Q", title="Fallzahl"),
    color=alt.Color("Typ:N", 
                    scale=alt.Scale(scheme="dark2"),
                    legend=alt.Legend(
        orient="none", legendX=20, legendY=20,
        fillColor="white", strokeColor="black", cornerRadius=7, padding=6,
        title="Gruppen"))
)
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['Tag'], empty='none')
selectors = alt.Chart(seir_data).mark_point().encode(
    alt.X("Tag", title="Tag"),
    opacity=alt.value(0),
).add_selection(nearest)
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)
text = line.mark_text(align='left', dx=4, dy=-8).encode(
    text=alt.condition(nearest, 'Fallzahl:Q', alt.value(' '))
)
rules = alt.Chart(seir_data).mark_rule(color='gray').encode(
    alt.X("Tag", title="Tag"),
).transform_filter(nearest)
alt.layer(
    line, selectors, points, rules, text
).properties(width=830, height=400)

alt.LayerChart(...)